In [1]:
import numpy as np
import scipy as sp
import math as ma
import qutip as qt
import matplotlib.pyplot as plt
import miscfuncs as mf
import pyplotsetup
%matplotlib inline

In [2]:
fockdim = 200

# Square GKP code stabiliser generators
alpha = np.sqrt(0.5*np.pi)
beta = 1j*alpha 
Sx = mf.displacement_fock(2*alpha, fockdim)
Sz = mf.displacement_fock(2*beta, fockdim)

Mx, My, Mz = mf.gkp_measure_ops(fockdim, alpha, beta) # Pauli-bin operators
a = qt.destroy(fockdim) # annihilation operator

In [3]:
def decode(state):
    if state.type == 'ket':
        rho = state*state.dag()
    else:
        rho = state
    return 0.5*(qt.qeye(2) + qt.expect(Mx, state)*qt.sigmax() + qt.expect(My,state)*qt.sigmay() 
            + qt.expect(Mz,state)*qt.sigmaz())

def squeezing(state, dB=True):
    if state.type == 'ket':
        rho = state*state.dag()
    else:
        rho = state
    Deltax = np.sqrt(-np.log(np.abs((Sx*rho).tr())**2)/(2*np.pi))
    Deltaz = np.sqrt(-np.log(np.abs((Sz*rho).tr())**2)/(2*np.pi))
    Delta = 0.5*(Deltax + Deltaz)
    if dB == True:
        return -10*np.log10(Delta**2)
    else:
        return Delta

In [4]:
def cost_function(A, *args):
    N = len(A) - 1
    w0 = args[0]
    J = args[1]
    T = 2*ma.pi/w0
    
    # Define driving function
    def f(t, args):
        A = args['A']
        N = args['N']
        w = args['w0']
        f = 0.5*A[0]
        for n in np.arange(1, N+1):
            f += A[n]*np.cos(4*n*w*t)
        return f

    # Compute Floquet operator
    H0 = w0*a.dag()*a
    H1 = -J*0.5*(Sz + Sz.dag())
    H = [H0, [H1, f]]
    options = qt.Options(nsteps = 1e4)
    U = qt.propagator(H, T, args = {'A': A, 'N': N, 'w0': w0}, options=options)

    # Floquet states and squeezings
    floquet_states, quasienergies = qt.floquet_modes(H, T, U=U)
    squeezings = []
    for i in range(len(floquet_states)):
        squeezings.append(squeezing(floquet_states[i]))
        
    return -max(squeezings)

In [5]:
N = 4
A0 = np.ones(N+1) # Initial guess for amplitudes: a_n = 1
w0 = 1
J = 0.01
result = sp.optimize.minimize(cost_function, A0, args = (w0,J) , method = 'L-BFGS-B')

In [6]:
result

      fun: -13.13386398095837
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.42498414, -0.14784494,  0.23809097,  0.11688321,  0.27563498])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 312
      nit: 20
   status: 0
  success: True
        x: array([1.2153989 , 1.01305987, 0.52979617, 1.18274339, 1.31876503])